In [1]:
!pip3 install PyGithub pandas > /dev/null 2>&1


In [2]:
# Define required imports
import numpy as np
import pandas as pd
import re
import sys

# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

<a id="cell1"></a>

### 1. Read the Raw Data

We start by reading in the raw dataset, displaying the first few rows of the dataframe, and taking a look at the columns and column types present.

In [ ]:
raw_data = pd.read_csv('jfk_weather.csv',
                       parse_dates=['DATE'])
raw_data.head()

In [ ]:
raw_data.dtypes

<a id="cell2"></a>

### 2. Clean the Data

As you can see above, there are a lot of fields which are non-numerical - usually these will be fields that contain text or categorical data, e.g. `HOURLYSKYCONDITIONS`.


In [5]:
# Choose what columns to import from raw data
column_subset = [
    'DATE',
    'HOURLYVISIBILITY',
    'HOURLYDRYBULBTEMPF',
    'HOURLYWETBULBTEMPF',
    'HOURLYDewPointTempF',
    'HOURLYRelativeHumidity',
    'HOURLYWindSpeed',
    'HOURLYWindDirection',
    'HOURLYStationPressure',
    'HOURLYPrecip',
    'HOURLYAltimeterSetting',
    'HOURLYPressureTendency'
]

# Filter dataset to relevant columns
hourly_data = raw_data[column_subset]
# Set date index
hourly_data = hourly_data.set_index(pd.DatetimeIndex(hourly_data['DATE']))
hourly_data.drop(['DATE'], axis=1, inplace=True)
hourly_data.replace(to_replace='*', value=np.nan, inplace=True)
hourly_data.head()

,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPrecip,HOURLYAltimeterSetting,HOURLYPressureTendency
DATE,,,,,,,,,,,
2010-01-01 00:51:00,6.00,33.0,32.0,31,92.0,0.0,000,29.97,0.01,29.99,8.0
2010-01-01 01:00:00,5.59,33.0,32.0,31,92.0,0.0,000,29.96,NaN,NaN,8.0
2010-01-01 01:51:00,6.00,33.0,33.0,32,96.0,0.0,000,29.97,0.02,29.99,NaN
2010-01-01 02:03:00,6.00,34.0,33.0,32,93.0,0.0,000,29.97,T,29.99,NaN
2010-01-01 02:28:00,5.00,34.0,33.0,32,93.0,0.0,000,29.97,T,29.99,NaN


In [6]:
hourly_data.dtypes

HOURLYVISIBILITY           object
HOURLYDRYBULBTEMPF         object
HOURLYWETBULBTEMPF         object
HOURLYDewPointTempF        object
HOURLYRelativeHumidity     object
HOURLYWindSpeed            object
HOURLYWindDirection        object
HOURLYStationPressure      object
HOURLYPrecip               object
HOURLYAltimeterSetting     object
HOURLYPressureTendency    float64
dtype: object

In [ ]:
hourly_data['HOURLYPrecip'].unique()

In [8]:
# Fix imported data
hourly_data['HOURLYPrecip'].replace(to_replace='T', value='0.00', inplace=True)
hourly_data['HOURLYPrecip'].replace('0.020.01s', np.nan, inplace=True)

In [9]:
# Set of columns to convert
messy_columns = column_subset[1:]

# Convert columns to float32 datatype
for i in messy_columns:
    hourly_data[i] = hourly_data[i].apply(
        lambda x: re.sub('[^0-9,.-]', '', x)
        if type(x) == str else x).replace('', np.nan).astype(('float32'))

In [ ]:
print(hourly_data.info())
print()
hourly_data.head()

In [11]:
# Generate the summary statistics for each column
hourly_data.describe()

,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPrecip,HOURLYAltimeterSetting,HOURLYPressureTendency
count,96464.000000,111403.000000,111245.000000,111395.000000,111395.000000,111358.000000,110444.000000,111256.000000,79762.000000,86878.000000,48038.000000
mean,8.758805,55.310612,49.703098,43.309196,67.132141,11.268045,192.511597,29.990641,0.007032,30.009533,4.388963
std,2.742651,17.169744,16.062113,19.357059,20.278572,6.175006,107.606514,0.238452,0.042636,0.239821,2.741795
min,0.000000,1.000000,-1.000000,-22.000000,8.000000,0.000000,0.000000,28.490000,0.000000,28.510000,0.000000
25%,9.940000,42.000000,37.000000,29.000000,51.000000,7.000000,110.000000,29.850000,0.000000,29.870001,2.000000
50%,10.000000,56.000000,50.000000,45.000000,69.000000,10.000000,200.000000,29.990000,0.000000,30.010000,5.000000
75%,10.000000,70.000000,64.000000,60.000000,85.000000,15.000000,280.000000,30.139999,0.000000,30.160000,7.000000
max,99.419998,102.000000,85.000000,84.000000,100.000000,53.000000,360.000000,30.830000,2.410000,30.850000,8.000000


In [12]:
# Check if categorical variable HOURLYPressureTendency ever has a non-integer
# entry outside the bounds of 0-8
cond =\
    len(hourly_data[~hourly_data['HOURLYPressureTendency'].isin(
        list(range(0, 9)) + [np.nan])])

print('Hourly Pressure Tendency should be between 0 and 8: {}'
      .format(cond == 0))

Hourly Pressure Tendency should be between 0 and 8: True


In [13]:
# Hourly Visibility should be between 0 and 10
hourly_data[(hourly_data['HOURLYVISIBILITY'] < 0) | (hourly_data['HOURLYVISIBILITY'] > 10)]

,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPrecip,HOURLYAltimeterSetting,HOURLYPressureTendency
DATE,,,,,,,,,,,
2011-10-16 11:51:00,14.000000,68.0,53.0,37.0,33.0,16.0,250.0,29.85,NaN,29.870001,NaN
2015-06-21 17:45:00,99.419998,79.0,72.0,68.0,70.0,37.0,310.0,29.74,0.0,29.760000,NaN


In [14]:
# Replace any hourly visibility figure outside these bounds with nan
hourly_data.loc[hourly_data['HOURLYVISIBILITY'] > 10, 'HOURLYVISIBILITY'] = np.nan

# Hourly Visibility should be between 0 and 10
cond = len(hourly_data[(hourly_data['HOURLYVISIBILITY'] < 0) | (hourly_data['HOURLYVISIBILITY'] > 10)])

print('Hourly Visibility should be between 0 and 10: {}'.format(cond == 0))

Hourly Visibility should be between 0 and 10: True


In [15]:
cond = len(hourly_data[hourly_data.index.duplicated()].sort_index())
print('Date index contains no duplicate entries: {}'.format(cond == 0))

Date index contains no duplicate entries: True


In [16]:
# Make sure time index is sorted and increasing
print('Date index is strictly increasing: {}'
      .format(hourly_data.index.is_monotonic_increasing))

Date index is strictly increasing: True


In [17]:
# Resample (downsample) to hourly rows (we're shifting everything up by 9 minutes!)
hourly_data = hourly_data.resample('60min').last().shift(periods=1)  # noqa Note: use resample('60min', base=51) to resample on the 51st of every hour

In [18]:
hourly_data['HOURLYPressureTendency'] =\
    hourly_data['HOURLYPressureTendency'].fillna(method='ffill')  # fill with last valid observation
hourly_data = hourly_data.interpolate(method='linear')  # interpolate missing values
hourly_data.drop(hourly_data.index[0], inplace=True)  # drop first row

In [19]:
print(hourly_data.info())
print()
hourly_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75119 entries, 2010-01-01 01:00:00 to 2018-07-27 23:00:00
Freq: 60T
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   HOURLYVISIBILITY        75119 non-null  float32
 1   HOURLYDRYBULBTEMPF      75119 non-null  float32
 2   HOURLYWETBULBTEMPF      75119 non-null  float32
 3   HOURLYDewPointTempF     75119 non-null  float32
 4   HOURLYRelativeHumidity  75119 non-null  float32
 5   HOURLYWindSpeed         75119 non-null  float32
 6   HOURLYWindDirection     75119 non-null  float32
 7   HOURLYStationPressure   75119 non-null  float32
 8   HOURLYPrecip            75119 non-null  float32
 9   HOURLYAltimeterSetting  75119 non-null  float32
 10  HOURLYPressureTendency  75119 non-null  float32
dtypes: float32(11)
memory usage: 3.7 MB
None



,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYWETBULBTEMPF,HOURLYDewPointTempF,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYStationPressure,HOURLYPrecip,HOURLYAltimeterSetting,HOURLYPressureTendency
DATE,,,,,,,,,,,
2010-01-01 01:00:00,6.0,33.0,32.0,31.0,92.0,0.0,0.0,29.969999,0.01,29.990000,8.0
2010-01-01 02:00:00,6.0,33.0,33.0,32.0,96.0,0.0,0.0,29.969999,0.02,29.990000,8.0
2010-01-01 03:00:00,5.0,33.0,33.0,32.0,96.0,0.0,0.0,29.969999,0.00,29.990000,8.0
2010-01-01 04:00:00,5.0,33.0,33.0,32.0,96.0,0.0,0.0,29.950001,0.00,29.969999,8.0
2010-01-01 05:00:00,5.0,33.0,32.0,31.0,92.0,0.0,0.0,29.930000,0.00,29.950001,8.0


In [20]:
# Transform HOURLYWindDirection into a cyclical variable using sin and cos transforms
hourly_data['HOURLYWindDirectionSin'] = np.sin(hourly_data['HOURLYWindDirection'] * (2. * np.pi / 360))
hourly_data['HOURLYWindDirectionCos'] = np.cos(hourly_data['HOURLYWindDirection'] * (2. * np.pi / 360))
hourly_data.drop(['HOURLYWindDirection'], axis=1, inplace=True)

In [21]:
# Transform HOURLYPressureTendency into 3 dummy variables based on NOAA documentation
hourly_data['HOURLYPressureTendencyIncr'] =\
    [1.0 if x in [0, 1, 2, 3]
        else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 0 through 3 indicates an increase in pressure over previous 3 hours
hourly_data['HOURLYPressureTendencyDecr'] =\
    [1.0 if x in [5, 6, 7, 8]
     else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 5 through 8 indicates a decrease over previous 3 hours
hourly_data['HOURLYPressureTendencyConst'] =\
    [1.0 if x == 4
     else 0.0 for x in hourly_data['HOURLYPressureTendency']]  # noqa 4 indicates no change during previous 3 hours
hourly_data.drop(['HOURLYPressureTendency'], axis=1, inplace=True)
hourly_data['HOURLYPressureTendencyIncr'] =\
    hourly_data['HOURLYPressureTendencyIncr'].astype(('float32'))
hourly_data['HOURLYPressureTendencyDecr'] =\
    hourly_data['HOURLYPressureTendencyDecr'].astype(('float32'))
hourly_data['HOURLYPressureTendencyConst'] =\
    hourly_data['HOURLYPressureTendencyConst'].astype(('float32'))

In [ ]:
hourly_data.columns

In [23]:
# define the new column names
columns_new_name = [
    'visibility',
    'dry_bulb_temp_f',
    'wet_bulb_temp_f',
    'dew_point_temp_f',
    'relative_humidity',
    'wind_speed',
    'station_pressure',
    'sea_level_pressure',
    'precip',
    'altimeter_setting',
    'wind_direction_sin',
    'wind_direction_cos',
    'pressure_tendency_incr',
    'pressure_tendency_decr',
    'pressure_tendency_const'
]

columns_name_map =\
    {c: columns_new_name[i] for i, c in enumerate(hourly_data.columns)}

hourly_data_renamed = hourly_data.rename(columns=columns_name_map)

In [ ]:
print(hourly_data_renamed.info())
print()
hourly_data_renamed.head()

In [25]:
# Explore some general information about the dataset
print('# of megabytes held by dataframe: {}'.format(
      str(round(sys.getsizeof(hourly_data_renamed) / 1000000, 2))))
print('# of features: {}'.format(str(hourly_data_renamed.shape[1])))
print('# of observations: {}'.format(str(hourly_data_renamed.shape[0])))
print('Start date: {}'.format(str(hourly_data_renamed.index[0])))
print('End date: {}'.format(str(hourly_data_renamed.index[-1])))
print('# of days: {}'.format(
      str((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days)))
print('# of months: {}'.format(
      str(round((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days / 30, 2))))
print('# of years: {}'.format(
      str(round((hourly_data_renamed.index[-1] - hourly_data_renamed.index[0]).days / 365, 2))))

# of megabytes held by dataframe: 4.81
# of features: 14
# of observations: 75119
Start date: 2010-01-01 01:00:00
End date: 2018-07-27 23:00:00
# of days: 3129
# of months: 104.3
# of years: 8.57


<a id="cell10"></a>

### 3. Save the Cleaned Data

Finally, we save the cleaned dataset as a Project asset for later re-use. You should see an output like the one below if successful:

```
{'file_name': 'jfk_weather_cleaned.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'jfkweatherdata-donotdelete-pr-...',
 'asset_id': '...'}
```

**Note**: In order for this step to work, your project token (see the first cell of this notebook) must have `Editor` role. By default this will overwrite any existing file.

In [27]:
hourly_data_renamed.to_csv(
    "jfk_weather_cleaned.csv",
    float_format='%g')